In [14]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder  # LabelEncoder 임포트

# 데이터 읽기
data = pd.read_csv('preprocessed_data.csv')

# 데이터 정렬
data_sorted = data.sort_values(by=['RID', 'VISCODE'])

# y와 X 정의
y = data_sorted['DX']
X = data_sorted.drop(['RID', 'VISCODE', 'DX_bl', 'DX'], axis=1)

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=data['DX'], random_state=42)

# X_train과 X_test를 PyTorch 텐서로 변환
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)

# LabelEncoder로 y_train과 y_test를 숫자형으로 인코딩
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# 인코딩된 값을 PyTorch 텐서로 변환
y_train = torch.tensor(y_train_encoded, dtype=torch.long)
y_test = torch.tensor(y_test_encoded, dtype=torch.long)


In [21]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.rnn(x)  # out: (batch_size, seq_len, hidden_size)
        # out의 차원이 2D라면 3D로 확장
        if out.dim() == 2:
            out = out.unsqueeze(1)  # (batch_size, 1, hidden_size)
        out = out[:, -1, :]  # 마지막 시점의 출력만 사용
        out = self.fc(out)   # Linear layer: (batch_size, output_size)
        return out


In [25]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.gru(x)  # out: (batch_size, seq_len, hidden_size)
        # out의 차원이 2D라면 3D로 확장
        if out.dim() == 2:
            out = out.unsqueeze(1)  # (batch_size, 1, hidden_size)
        out = out[:, -1, :]  # 마지막 시점의 출력만 사용
        out = self.fc(out)   # Linear layer: (batch_size, output_size)
        return out


In [27]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)  # out: (batch_size, seq_len, hidden_size)
        # out의 차원이 2D라면 3D로 확장
        if out.dim() == 2:
            out = out.unsqueeze(1)  # (batch_size, 1, hidden_size)
        out = out[:, -1, :]  # 마지막 시점의 출력만 사용
        out = self.fc(out)   # Linear layer: (batch_size, output_size)
        return out


In [22]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# 평가 함수 정의
def evaluate_model(model, X_test, y_test):
    model.eval()  # 평가 모드
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs, 1)
    
    # 정확도, F1 점수, 정밀도, 재현율 계산
    accuracy = accuracy_score(y_test, predicted)
    f1_macro = f1_score(y_test, predicted, average='macro')
    f1_micro = f1_score(y_test, predicted, average='micro')
    precision_macro = precision_score(y_test, predicted, average='macro')
    precision_micro = precision_score(y_test, predicted, average='micro')
    recall_macro = recall_score(y_test, predicted, average='macro')
    recall_micro = recall_score(y_test, predicted, average='micro')

    print(f'Accuracy: {accuracy}')
    print(f'F1 Score (Macro): {f1_macro}')
    print(f'F1 Score (Micro): {f1_micro}')
    print(f'Precision (Macro): {precision_macro}')
    print(f'Precision (Micro): {precision_micro}')
    print(f'Recall (Macro): {recall_macro}')
    print(f'Recall (Micro): {recall_micro}')


In [23]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=10, batch_size=32):
    criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류를 위한 손실 함수
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # 훈련 루프
    for epoch in range(epochs):
        model.train()  # 훈련 모드
        optimizer.zero_grad()  # 경사 초기화

        # 배치 처리 (전체 데이터를 배치 사이즈로 나누기)
        for i in range(0, len(X_train), batch_size):
            inputs = X_train[i:i+batch_size]
            targets = y_train[i:i+batch_size]

            # 순방향 전파
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # 역방향 전파
            loss.backward()
            optimizer.step()

        # 에폭마다 성능 평가
        if (epoch+1) % 2 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")
            evaluate_model(model, X_test, y_test)

# 모델 인스턴스 생성 및 훈련
input_size = X_train.shape[1]  # 입력 크기
output_size = len(y.unique())  # 클래스 수 (진단 유형)
hidden_size = 64  # 은닉층 크기

# 모델 선택 (RNN, GRU, LSTM 중 하나 선택)
model = RNNModel(input_size, hidden_size, output_size)

# 모델 훈련
train_model(model, X_train, y_train, X_test, y_test, epochs=10)


Epoch [2/10], Loss: 2.7659
Accuracy: 0.5404040404040404
F1 Score (Macro): 0.16374227433705946
F1 Score (Micro): 0.5404040404040404
Precision (Macro): 0.1354017896733876
Precision (Micro): 0.5404040404040404
Recall (Macro): 0.21479229989868287
Recall (Micro): 0.5404040404040404


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [4/10], Loss: 4.3447
Accuracy: 0.3595959595959596
F1 Score (Macro): 0.13119213148530737
F1 Score (Micro): 0.3595959595959596
Precision (Macro): 0.10686679844112233
Precision (Micro): 0.3595959595959596
Recall (Macro): 0.2119779353821907
Recall (Micro): 0.3595959595959596


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [6/10], Loss: 2.0423
Accuracy: 0.2636363636363636
F1 Score (Macro): 0.10762366044657158
F1 Score (Micro): 0.2636363636363636
Precision (Macro): 0.20268155645347596
Precision (Micro): 0.2636363636363636
Recall (Macro): 0.20754493094918627
Recall (Micro): 0.2636363636363636


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [8/10], Loss: 1.6800
Accuracy: 0.4535353535353535
F1 Score (Macro): 0.1500982563666795
F1 Score (Micro): 0.4535353535353535
Precision (Macro): 0.2326943278455577
Precision (Micro): 0.4535353535353535
Recall (Macro): 0.21287806491302563
Recall (Micro): 0.4535353535353535


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [10/10], Loss: 2.2386
Accuracy: 0.4535353535353535
F1 Score (Macro): 0.1500982563666795
F1 Score (Micro): 0.4535353535353535
Precision (Macro): 0.2326943278455577
Precision (Micro): 0.4535353535353535
Recall (Macro): 0.21287806491302563
Recall (Micro): 0.4535353535353535


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
model_gru = GRUModel(input_size, hidden_size, output_size)

# 모델 훈련
train_model(model_gru, X_train, y_train, X_test, y_test, epochs=10)

Epoch [2/10], Loss: 1.4735
Accuracy: 0.5616161616161616
F1 Score (Macro): 0.1824500086720597
F1 Score (Micro): 0.5616161616161616
Precision (Macro): 0.19310417731470364
Precision (Micro): 0.5616161616161616
Recall (Macro): 0.2226244077572761
Recall (Micro): 0.5616161616161616


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [4/10], Loss: 1.0668
Accuracy: 0.5626262626262626
F1 Score (Macro): 0.17776191628249322
F1 Score (Micro): 0.5626262626262626
Precision (Macro): 0.15824434274091026
Precision (Micro): 0.5626262626262626
Recall (Macro): 0.2222222222222222
Recall (Micro): 0.5626262626262626


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [6/10], Loss: 1.3757
Accuracy: 0.4696969696969697
F1 Score (Macro): 0.17803837121300783
F1 Score (Micro): 0.4696969696969697
Precision (Macro): 0.1830346399399782
Precision (Micro): 0.4696969696969697
Recall (Macro): 0.2164318715921082
Recall (Micro): 0.4696969696969697


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [8/10], Loss: 1.4077
Accuracy: 0.5646464646464646
F1 Score (Macro): 0.1800761071243675
F1 Score (Micro): 0.5646464646464646
Precision (Macro): 0.2703216374269006
Precision (Micro): 0.5646464646464646
Recall (Macro): 0.22314430613185798
Recall (Micro): 0.5646464646464646


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [10/10], Loss: 1.1478
Accuracy: 0.5161616161616162
F1 Score (Macro): 0.20450516089860352
F1 Score (Micro): 0.5161616161616162
Precision (Macro): 0.19292170374927134
Precision (Micro): 0.5161616161616162
Recall (Macro): 0.22049856992913408
Recall (Micro): 0.5161616161616162


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [28]:
model_lstm = LSTMModel(input_size, hidden_size, output_size)

# 모델 훈련
train_model(model_lstm, X_train, y_train, X_test, y_test, epochs=10)

Epoch [2/10], Loss: 1.3119
Accuracy: 0.32626262626262625
F1 Score (Macro): 0.1267097606417932
F1 Score (Micro): 0.32626262626262625
Precision (Macro): 0.11725632281898266
Precision (Micro): 0.32626262626262625
Recall (Macro): 0.19980506822612087
Recall (Micro): 0.32626262626262625


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [4/10], Loss: 1.2294
Accuracy: 0.5626262626262626
F1 Score (Macro): 0.17486733001658375
F1 Score (Micro): 0.5626262626262626
Precision (Macro): 0.15176404274654612
Precision (Micro): 0.5626262626262626
Recall (Macro): 0.2222222222222222
Recall (Micro): 0.5626262626262626


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [6/10], Loss: 0.9970
Accuracy: 0.5616161616161616
F1 Score (Macro): 0.17424523390392413
F1 Score (Micro): 0.5616161616161616
Precision (Macro): 0.1506492256803488
Precision (Micro): 0.5616161616161616
Recall (Macro): 0.22188449848024316
Recall (Micro): 0.5616161616161616


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [8/10], Loss: 0.9986
Accuracy: 0.5262626262626262
F1 Score (Macro): 0.21110453397283957
F1 Score (Micro): 0.5262626262626262
Precision (Macro): 0.22160285000611823
Precision (Micro): 0.5262626262626262
Recall (Macro): 0.2252210969435301
Recall (Micro): 0.5262626262626262


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [10/10], Loss: 1.0349
Accuracy: 0.46565656565656566
F1 Score (Macro): 0.17243194560163036
F1 Score (Micro): 0.46565656565656566
Precision (Macro): 0.17727240765477298
Precision (Micro): 0.46565656565656566
Recall (Macro): 0.22278462801107807
Recall (Micro): 0.46565656565656566


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Why was micro or macro averaging used for performance metrics?

micro averaging은 data point가 더 많은 클래스에 더 가중치를 주고 싶을 때 이용. works for imbalanced datasets where certain classes may have significantly more samples than others

macro averaging은 모든 클래스를 동일하게 취급할 때 이용. 

-micro가 더 잘 나옴. 특정 class가 더 data point 많았기 때문일 듯.